# OutLine
1. Introduction to Random Quantum Circuits
2. Projected Entangled Pair States Method
3. Complexity Analysis
4. Numerical Results

In [1]:
include("../find_meteor.jl")
using Meteor.TensorNetwork
using Meteor.QuantumCircuit

# Introduction to Random Quantum Circuits
    
1. Finding a problem which is very hard to solve with the best supercomputer in the world, but can be efficiently solved with a quantum computer.
2. Random quantum circuit (**RQCs**) sampling problem is a standard problem used to demonstrate quantum supremacy in near term.
3. We propose to use Projected Entangled Pair States (**PEPS**) to study large-scale RQCs.

The random quantum circuit proposed in [Characterizing Quantum Supremacy in Near-Term Devices](arXiv:1608.00263v3)
![google circuit](GoogleCircuitH.png)

Rules for $L_h\times L_v$ circuit:
1. Apply a Hadamard gate to each qubit to initialize the qubits to a symmetric superposition.
2. Apply controlled-phase (CZ) gates alternating between $8$ configurations as above entangle neighbouring qubits.
3. Apply a randomly chosen gate ($T$, $\sqrt{X}$ or $\sqrt{Y}$) to each qubit on which the CZ gates has not just been applied, according to certain rules.
4. Repeat steps 2 and 3 to add layers of depth to the circuit.
5. Apply a final Hadamard gate to each qubit.

In [2]:
function google_circuit(m::Int, n::Int, depth::Int)
    XYT = Dict(0=>sqrtX, 1=>sqrtY, 2=>T)
    Xoperated = zeros(Bool, m, n)
    Yoperated = zeros(Bool, m, n)
    Toperated = zeros(Bool, m, n)
    CZoperated = zeros(Bool, m, n)

    ControlZ = CZ
    config1 = []
    config2 = []
    config3 = []
    config4 = []
    config5 = []
    config6 = []
    config7 = []
    config8 = []
    config1sites = []
    config2sites = []
    config3sites = []
    config4sites = []
    config5sites = []
    config6sites = []
    config7sites = []
    config8sites = []
    toind(i,j) = linear_index((m, n), (i, j), false)

    # 1 configuration
    for jj in 0:2:(n-2)
        for ii in ((div(jj,2)+1)%2):2:(m-1)
            i = ii + 1
            j = jj + 1
            push!(config1, ((toind(i, j), toind(i, j+1)), ControlZ))
            push!(config1sites, (i, j))
            push!(config1sites, (i, j+1))
        end
    end

    # 2 configuration
    for jj in 0:2:(n-2)
        for ii in ((div(jj,2))%2):2:(m-1)
            i = ii + 1
            j = jj + 1
            push!(config2, ((toind(i, j), toind(i, j+1)), ControlZ))
            push!(config2sites, (i, j))
            push!(config2sites, (i, j+1))
        end
    end

    # 5 configuration
    for jj in 1:2:(n-2)
        for ii in ((div(jj,2)+1)%2):2:(m-1)
            i = ii + 1
            j = jj + 1
            push!(config5, ((toind(i, j), toind(i, j+1)), ControlZ))
            push!(config5sites, (i, j))
            push!(config5sites, (i, j+1))
        end
    end

    # 6 configuration
    for jj in 1:2:(n-2)
        for ii in (((div(jj,2)))%2):2:(m-1)
            i = ii + 1
            j = jj + 1
            push!(config6, ((toind(i, j), toind(i, j+1)), ControlZ))
            push!(config6sites, (i, j))
            push!(config6sites, (i, j+1))
        end
    end

    # 3 configuration
    for ii in 1:2:(m-2)
        for jj in ((div(ii,2)+1)%2):2:(n-1)
            i = ii + 1
            j = jj + 1
            push!(config3, ((toind(i, j), toind(i+1, j)), ControlZ))
            push!(config3sites, (i, j))
            push!(config3sites, (i+1, j))
        end
    end

    # 4 configuration
    for ii in 1:2:(m-2)
        for jj in ((div(ii,2))%2):2:(n-1)
            i = ii + 1
            j = jj + 1
            push!(config4, ((toind(i, j), toind(i+1, j)), ControlZ))
            push!(config4sites, (i, j))
            push!(config4sites, (i+1, j))
        end
    end

    # 7 configuration
    for ii in 0:2:(m-2)
        for jj in ((div(ii,2))%2):2:(n-1)
            i = ii + 1
            j = jj + 1
            push!(config7, ((toind(i, j), toind(i+1, j)), ControlZ))
            push!(config7sites, (i, j))
            push!(config7sites, (i+1, j))
        end
    end

    # 8 configuration
    for ii in 0:2:(m-2)
        for jj in ((div(ii,2)+1)%2):2:(n-1)
            i = ii + 1
            j = jj + 1
            push!(config8, ((toind(i, j), toind(i+1, j)), ControlZ))
            push!(config8sites, (i, j))
            push!(config8sites, (i+1, j))
        end
    end

    configs=[(config1, config1sites), (config2, config2sites), (config3, config3sites),
        (config4, config4sites), (config5, config5sites), (config6, config6sites),
        (config7, config7sites), (config8, config8sites)]

    function __generate_one_body_gates()
        one_body_gates = []
        tmpXoperated = zeros(Bool, m, n)
        tmpYoperated = zeros(Bool, m, n)
        for i in 1:m
            for j in 1:n
                # Place a gate at qubit q only if this qubit is
                # occupied by a CZ gate in the previous cycle
                if CZoperated[i, j]
                    # Place a T gate at qubit q if there are no single
                    # qubit gates in the previous cycles at qubit q except
                    # for the initial cycle of Hadamard gates
                    if !Toperated[i, j]
                        push!(one_body_gates, (toind(i, j), XYT[2]))
                        Toperated[i, j] = true
                    else
                        # Any gate at qubit q should be different from
                        # the gate at qubit q in the previous cycle.
                        if tmpXoperated[i, j]
                            push!(one_body_gates, (toind(i, j), XYT[1]))
                            tmpYoperated[i, j] = true
                        else
                            if tmpYoperated[i, j]
                                push!(one_body_gates, (toind(i, j), XYT[0]))
                                tmpXoperated[i, j] = true
                            else
                                r = rand(0:1)
                                push!(one_body_gates, (toind(i, j), XYT[0]))
                                if r == 0
                                    tmpXoperated[i, j] = true
                                else
                                    tmpYoperated[i, j] = true
                                end
                            end
                        end
                    end
                end
            end
        end
        Xoperated = tmpXoperated
        Yoperated = tmpYoperated
        return one_body_gates
    end


    circuit = QCircuit()
    for i in 1:m
        for j in 1:n
            push!(circuit, (toind(i, j), H))
        end
    end
    for i in 0:(depth-1)
        present_config = i % 8 + 1
        # println("present config is $present_config")
        config, configsites = configs[present_config]
        fill!(CZoperated, false)
        for item in configsites
            CZoperated[item...] = true
        end
        one_body_gates = __generate_one_body_gates()
        for item in one_body_gates
            push!(circuit, item)
        end
        for item in config
            push!(circuit, item)
        end
        # append!(circuit, one_body_gates)
        # append!(circuit, config)
    end
    for i in 1:m
        for j in 1:n
            push!(circuit, (toind(i, j), H))
        end
    end
    return circuit
end

google_circuit (generic function with 1 method)

# Projected Entangled Pair States Method

Introduction to PEPS:
The PEPS-based quantum circuit simulator can be described as the following figure ![PEPS](FigPEPS.png)

# Complexity Analysis

The quantum state is stored as ![f1](psi.jpg)
1. Classical numerical complexity of gate operations $\mathcal{O}(\chi^4)$, with $\chi=\max\left({\rm dim}(l), {\rm dim}(r), {\rm dim}(u), {\rm dim}(d)\right)$
2. Classical numerical complexity of computing one amplitude ![f2](complexity.jpg)

Frontier of quantum supremacy ![f3](FigData.png)

Generating a $m\times n\times depth$ circuit

In [3]:
m = 8
n = 8
depth = 12

circuit = google_circuit(m, n, depth)
println("total number of gate operations $(length(circuit)).")

total number of gate operations 632.


**Gate fusion technique**-Absorbing one-body gate operations into two-body gate operations

In [4]:
println("total number of gates $(length(circuit)).")
circuit = fuse_gate(circuit)
println("number of gates after gate fusion $(length(circuit)).")

total number of gates 632.
number of gates after gate fusion 168.


Specifying the topology of the circuit

In [5]:
connectivities = square_connectivity(m, n)
println(connectivities)

[(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 31), (31, 32), (33, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (41, 42), (42, 43), (43, 44), (44, 45), (45, 46), (46, 47), (47, 48), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 64), (1, 9), (2, 10), (3, 11), (4, 12), (5, 13), (6, 14), (7, 15), (8, 16), (9, 17), (10, 18), (11, 19), (12, 20), (13, 21), (14, 22), (15, 23), (16, 24), (17, 25), (18, 26), (19, 27), (20, 28), (21, 29), (22, 30), (23, 31), (24, 32), (25, 33), (26, 34), (27, 35), (28, 36), (29, 37), (30, 38), (31, 39), (32, 40), (33, 41), (34, 42), (35, 43), (36, 44), (37, 45), (38, 46), (39, 47), (40, 48), (41, 49), (42, 50), (43, 51), (44, 52), (45, 53), (46, 54), (47,

Initializing the quantum state represented as a tensor network

In [6]:
state = GraphState(ComplexF32, connectivities)

GraphState{Complex{Float32}}({64, 112} undirected simple Int64 graph, IndexedTensor{Complex{Float32},N} where N[IndexedTensor{Complex{Float32},3}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (1, 2, 3)), IndexedTensor{Complex{Float32},4}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (4, 2, 5, 6)), IndexedTensor{Complex{Float32},4}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (7, 5, 8, 9)), IndexedTensor{Complex{Float32},4}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (10, 8, 11, 12)), IndexedTensor{Complex{Float32},4}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (13, 11, 14, 15)), IndexedTensor{Complex{Float32},4}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (16, 14, 17, 18)), IndexedTensor{Complex{Float32},4}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (19, 17, 20, 21)), IndexedTensor{Complex{Float32},3}(Complex{Float32}[1.0f0 + 0.0f0im; 0.0f0 + 0.0f0im], (22, 20, 23)), IndexedTensor{Complex{Float32},4}(Complex{Float32}[1.0f0 + 0.0f0im; 

Applying the circuit onto the quantum state

In [7]:
apply!(circuit, state)

168-element Array{Tuple{Int64,Float64},1}:
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 (2, 0.0)
 ⋮
 (10, 1.875618301148377e-15)
 (8, 2.528081639429497e-15)
 (10, 2.2304984245100903e-15)
 (6, 0.0)
 (10, 4.460998543086075e-15)
 (8, 3.006413003205262e-15)
 (10, 1.8756185129066138e-15)
 (6, 0.0)
 (4, 1.234073954137216e-10)
 (8, 0.0)
 (6, 0.0)
 (5, 2.5164171094105292e-11)

Computing amplitudes

In [8]:
L = m * n
basis = [rand(0:1, L) for i in 1:5]
amps = amplitudes(state, basis, scaling=sqrt(2))

5-element Array{Complex{Float32},1}:
 -0.44874358f0 - 0.014058113f0im
  0.07292746f0 + 0.043092266f0im
  -0.2113406f0 - 1.217452f0im
   -0.972785f0 - 0.8453204f0im
 -0.38062093f0 - 0.28016192f0im

Specifying a **contraction path**

For example, as ![f4](contraction_path.png)

In [9]:
using Meteor.QuantumCircuit: linear_index
rowmajor = false
contract_path = Int[]
sp = (m, n)
for i in 1:8
    for j in 1:4
        push!(contract_path, linear_index(sp, (i, j), rowmajor))
    end
end

for i in 8:-1:1
    for j in 5:8
        push!(contract_path, linear_index(sp, (i, j), rowmajor))
    end
end

println("contraction path $contract_path")
amps = amplitudes(state, basis, scaling=sqrt(2), cut=[(25, 33), (26, 34)], path=contract_path)

contraction path [1, 9, 17, 25, 2, 10, 18, 26, 3, 11, 19, 27, 4, 12, 20, 28, 5, 13, 21, 29, 6, 14, 22, 30, 7, 15, 23, 31, 8, 16, 24, 32, 40, 48, 56, 64, 39, 47, 55, 63, 38, 46, 54, 62, 37, 45, 53, 61, 36, 44, 52, 60, 35, 43, 51, 59, 34, 42, 50, 58, 33, 41, 49, 57]


5-element Array{Complex{Float64},1}:
 -0.44874350400304563 - 0.01405791410866053im
  0.07292721219510856 + 0.04309175396405163im
 -0.21134013127713946 - 1.2174510827983864im
  -0.9727855025067544 - 0.8453202836485803im
  -0.3806203092880014 - 0.2801625288144569im

# Results on $8\times 8 \times 25$ circuits

Small-scale implementation, showing Proter-Thomas distribution for $8\times 8$ circuit of $25$ depth with $10000$ random samples ![f5](gumbel.png)

In [10]:
depth = 24

circuit = google_circuit(m, n, depth)
println("total number of gate operations $(length(circuit)).")

state = GraphState(ComplexF32, connectivities)
apply!(circuit, state)
amps = amplitudes(state, basis, scaling=sqrt(2), cut=[(25, 33), (26, 34), (27, 35)], path=contract_path)

total number of gate operations 1136.


InterruptException: InterruptException: